In [ ]:
import os.path
import pandas as pd


from sklearn.feature_extraction.text import CountVectorizer
import requests

In [ ]:
source = pd.read_csv("data/langmeta.csv").fillna(value = "")
stop_words_german = []
with open("data/stop_words_german.txt") as file:
    for line in file:
        stop_words_german.append(line.replace('\n', '').encode('ASCII','ignore').decode('ASCII'))

In [ ]:
if (os.path.exists("data/keywordsmeta.csv")):
    os.remove("data/keywordsmeta.csv")

In [ ]:

german_vec = CountVectorizer(
    ngram_range=(1, 1), 
    stop_words=stop_words_german,
    max_features=5,
    strip_accents='ascii'
)
english_vec = CountVectorizer(
    ngram_range=(1, 1), 
    stop_words="english",
    max_features=5,
    strip_accents='ascii'
)


In [ ]:

text=[]
for index, row in source.iterrows():  
    metadict = {"Identifier":row["Identifier"]}      
    if(row['Language'] != ""):
        if(int(row['Textsize']) > 100):
            if(row['Language'] == "GERMAN"):
                textpfad = "temp/" + row["Identifier"] + ".txt"
                headertextpfad = "temp/" + row["Identifier"] + "_header.txt"
                with open(textpfad, 'r', encoding="utf-8") as textfile:
                    german_vec.fit_transform(textfile)
                    metadict.update({"Topwords":str(german_vec.get_feature_names_out())}) 
                try:
                    with open(headertextpfad, 'r', encoding="utf-8") as textfile:
                        german_vec.fit_transform(textfile)
                        metadict.update({"Header_Topwords":str(german_vec.get_feature_names_out())})
                except:
                    print("Error with Headertext")
            elif(row['Language'] == "ENGLISH"):
                textpfad = "temp/" + row["Identifier"] + ".txt"
                with open(textpfad, 'r', encoding="utf-8") as textfile:
                    english_vec.fit_transform(textfile)                    
                    metadict.update({"Topwords":str(english_vec.get_feature_names_out())}) 
                try:
                    with open(headertextpfad, 'r', encoding="utf-8") as textfile:
                        english_vec.fit_transform(textfile)                    
                        metadict.update({"Header_Topwords":str(english_vec.get_feature_names_out())}) 
                except:
                    print("Error with Headertext")
            else:
                frame = pd.DataFrame(metadict, [0])
                if (os.path.exists("data/keywordsmeta.csv")):   
                    newframe = pd.concat([oldframe, frame])
                    newframe.to_csv("data/keywordsmeta.csv", index=False)
                    oldframe = newframe
                else:
                    frame.to_csv("data/keywordsmeta.csv", index=False)
                    oldframe = frame
                metadict.clear
                topwords=""
                print("finished: ", row["Identifier"], " couldn't match language")
                continue
         #ask lobid.org to find a fitting keyword 
            if("Header_Topwords" in metadict):
                wordcloud = str(metadict["Header_Topwords"]).replace("'", "")\
                                                            .removeprefix("[")\
                                                            .removesuffix("]")\
                                                            .split(" ")
            else:
                wordcloud = str(metadict["Topwords"]).replace("'", "")\
                                                     .removeprefix("[")\
                                                     .removesuffix("]")\
                                                     .split(" ")
            Keywordlist= list()
            KeyIDlist =list()
            for word in wordcloud:
                api_url = "https://lobid.org/gnd/search?q=preferredName:" + word + "&filter=type:SubjectHeading&format=json"
                response = requests.get(api_url)
                if(response.status_code == requests.codes.ok):
                    for member in response.json()["member"]: 
                        if('SubjectHeading' in member["type"] or 'SubjectHeadingSensoStricto' in member["type"]):
                            Keywordlist.append(member["preferredName"])
                            KeyIDlist.append(member["id"])
                            break

            metadict.update({"Keywordnames":str(Keywordlist)})       
            metadict.update({"KeywordIDs":str(KeyIDlist)})                           
                    
                    
                    
                    
                    
                    
    frame = pd.DataFrame(metadict, [0])
    if (os.path.exists("data/keywordsmeta.csv")):   
    #oldframe = pd.read_csv("data/langmeta.csv")
        newframe = pd.concat([oldframe, frame])
        newframe.to_csv("data/keywordsmeta.csv", index=False)
        oldframe = newframe
    else:
        frame.to_csv("data/keywordsmeta.csv", index=False)
        oldframe = frame
    metadict.clear
    topwords="" 
    print("finished: ", row["Identifier"])
    #break     
